In [1]:
import pandas as pd
import tensorflow as tf
import collections
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR, NuSVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import normalize

C:\Users\grave\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
COLUMN_TYPES = collections.OrderedDict([
    ("crime", float), # Important
    ("zn", float),
    ("indus", float),
    ("chas", int),
    ("nox", float),
    ("rm", float), # Important! Without this accuracy drops
    ("age", float),
    ("dis", float), # Important! 
    ("rad", int),
    ("tax", int),
    ("ptratio", float), # Important!
    ("b", float), # Doesnt do much
    ("lstat", float), # Important
    ("medv", float)
])

In [3]:
df = pd.read_csv('housingdata.csv', header=0, delimiter=';', names=COLUMN_TYPES.keys(), dtype=COLUMN_TYPES)
#df = df.drop([], axis=1) # Drop features.

In [35]:
seed = sum([ord(x) for x in "LIGHT FROM LIGHT"])
np.random.seed(seed)

In [37]:
# Training features, retain 0.9 for training, 0.1 for test (after cross validation and building of final model)
df_sampled = df.sample(frac=1, random_state=seed)
y_name = 'medv'
mean = np.mean(df["medv"])
df["medv"] = df["medv"].apply(lambda x: (x - mean))
# Features
x_train, x_test = np.split(df_sampled, [int(0.9*len(df_sampled))])
# Targets
y_train = x_train.pop(y_name)
y_test =  x_test.pop(y_name)

In [ ]:
#model = GradientBoostingRegressor(random_state=seed)
#param_grid = { "loss"              : ['ls', 'huber', 'lad'],
#               "n_estimators"      : [1,25,100,300,500],
#               "max_depth"         : [1,3,6,9,12,15,18],
#               "min_samples_split" : [2,4,6,8,10],
#               "learning_rate": [0.01, 0.05, 0.1]}
#grid_search = GridSearchCV(model, param_grid, n_jobs=-1, cv=2, scoring='r2', verbose=1)
#grid_search.fit(x_train, y_train)
#print(grid_search.best_params_)

In [23]:
#model = GradientBoostingRegressor(loss='huber', 
#                                 max_depth=3, 
#                                 min_samples_split=4, 
#                                 n_estimators=300,
#                                 learning_rate=0.1,
#                                 random_state=seed)
#
#scores = cross_val_score(model, x_train, y_train, cv=5)
#scores # array([0.88990719, 0.89271209, 0.88811618, 0.76994707, 0.87563016])

array([0.88990719, 0.89271209, 0.88811618, 0.76994707, 0.87563016])

In [38]:
model.fit(x_train, y_train)
y_pred = pd.Series(model.predict(x_test), index=y_test.index)
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred)) # Lower the better
print("Mean absolute error: %.2f" % mean_absolute_error(y_test, y_pred)) # Lower the better
print("Explained Variance Score: %.2f" % explained_variance_score(y_test, y_pred)) # Explained variance score: 1 is perfect
print('R2 score: %.2f' % r2_score(y_test, y_pred)) # 1 is perfect

Mean squared error: 6.20
Mean absolute error: 1.90
Explained Variance Score: 0.93
R2 score: 0.92


In [ ]:
#y_test_pred = pd.Series(regr.predict(x_test), index=y_test.index)
#print("Mean squared error: %.2f" % mean_squared_error(y_test, y_test_pred)) # Lower the better
#print("Mean absolute error: %.2f" % mean_absolute_error(y_test, y_test_pred)) # Lower the better
#print("Explained Variance Score: %.2f" % explained_variance_score(y_test, y_test_pred)) # Explained variance score: 1 is perfect
#print('R2 score: %.2f' % r2_score(y_test, y_test_pred)) # 1 is perfect